In [ ]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [ ]:
data_url = './fraud.csv'
df = pd.read_csv(data_url, sep=',')
df.head(20)

In [ ]:
# ID열을 삭제해줍니다.
del df["ID"] 

In [ ]:
# Label(Y_data)을 따로 저장해 줍니다.
Y_data = df.pop("Fraud")
# as_matrix()함수를 통해 array형태로 변환시켜 줍니다.
# Convert the frame to its Numpy-array representation.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.as_matrix.html

In [ ]:
Y_data = Y_data.as_matrix()
Y_data

In [ ]:
type(Y_data)

In [ ]:
df.head(20)

In [ ]:
a = df.History.unique()
b = np.array

In [ ]:
for i in a:
    b = np.append(b, 'History_'+i)

In [ ]:
b[1]

In [ ]:
dummy_var_History = pd.get_dummies(df.History)
dummy_var_History.rename(columns={'arrears': 'History_arrears', 'current': 'History_current', 'none': 'History_none', 'paid': 'History_paid'}, inplace=True)
dummy_var_History

In [ ]:
df.Accommodation.unique()

In [ ]:
dummy_var_CoApplicant = pd.get_dummies(df.CoApplicant)
dummy_var_CoApplicant.rename(columns={'none': 'CoApplicant_none', 'guarantor': 'CoApplicant_guarantor', 'coapplicant': 'CoApplicant_coapplicant'}, inplace=True)
dummy_var_CoApplicant

In [ ]:
dummy_var_Accommodation = pd.get_dummies(df.Accommodation)
dummy_var_Accommodation.rename(columns={'own': 'Accommodation_own', 'rent': 'Accommodation_rent', 'free': 'Accommodation_free'}, inplace=True)
dummy_var_Accommodation

In [ ]:
df_true = pd.concat([df.drop(['History'], axis=1),dummy_var_History], axis=1)
df_true = pd.concat([df_true.drop(['CoApplicant'], axis=1),dummy_var_CoApplicant], axis=1)
df_true = pd.concat([df_true.drop(['Accommodation'], axis=1),dummy_var_Accommodation], axis=1)

In [ ]:
x_df = df_true
x_df.head(30) # dummy변수로 변환

In [ ]:
x_data = x_df.as_matrix()
x_data

In [ ]:
Y_data == True # boolean index

In [ ]:
len(set(Y_data))

In [ ]:
P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True

In [ ]:
ix_Y_True = np.where(Y_data) # Y_data == True인 인덱스를 뽑아줍니다.

In [ ]:
ix_Y_False = np.where(Y_data==False)

In [ ]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0) / sum(Y_data==True))  # Q.뒤에 sum(Y_data == True) 필요한가요? # 앞에 식이 P(X_cap_Y1)인 것 같은데...
p_x_y_false = (x_data[ix_Y_False].sum(axis=0) / sum(Y_data==False))# y가 True라는 전제의 조건부 확률이기 때문입니다

In [ ]:
# 총 10개의 값에 대해서 확률을 구해준다.
x_test = [0,1,0,0,0,1,0, 0,1,0]
p_out = []

for n, i in enumerate(x_test):
    if i == 0:
        p_out.append(1-p_x_y_true[n])
    else:
        p_out.append(p_x_y_true[n])

In [ ]:
p_out

In [ ]:
p_out_x_y_t = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_true) ]
p_out_x_y_true = 1

for i in p_out_x_y_t:
    p_out_x_y_true*=i

In [ ]:
p_out_x_y_true

In [ ]:
[ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_false) ]
p_out_x_y_f = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_false) ]
p_out_x_y_false = 1
for i in p_out_x_y_f:
    p_out_x_y_false*=i
p_out_x_y_false

In [ ]:
import math

p_y_true_test = p_out_x_y_true/(p_out_x_y_true+p_out_x_y_false)
p_y_false_test = p_out_x_y_false/(p_out_x_y_true+p_out_x_y_false)

In [ ]:
# p_x_y_true의 해당 y의 값에대해 조건부 확률을 곱한게 우도인 ㅣikelihood이다
# 이 우도에 p(y)를 곱하고 p(Xi)의 곱들로 나누어 주면 원하는 확률을 얻게된다
# 이떄 우도는 독립변수 x가 많아 질 수록 확률분포가 넓어지므로 sparse해져 간혹 값이 매우 작아져 독립이라는 가정하에 각 변수 x의 조건부 확률을
# 곱하였을 때 값이 작아지는 경우가 생긴다
# 이때 보통 분모에 들어가는 p(Xi)의 곱들은 p(y|x)에서 y의 값에 상관없이 들어가므로 p(y)*likelihood의 값만 비교해도 되는데 그 값이 너무 작아지면
# 비교하기가 힘드므로 log를 취해주거나 값이 0에 수렴하거나 0이 되지않도록 smoothing을 통해 최소한의 확률을 정해준다
# 현재 주어진 iris데이터의 경우 5,8번쨰 돌립변수 x의 조건부확률이 1이 되어버려 0에 근사하는게 아닌 y가 false일때 아예 우도값이 0이된다
# 이는 smoothing을 통해서는 보정이 가능하지만 log를 취하게 되면 log(0)은 값이 없고 근사하더라도 음의 무한으로 발산하므로 데이터 비교에 용이하지않다
# 따라서 아래처럼 smoothing을 통해 확률을 구한다

In [ ]:
smoothing_p = 2

p_x_y_true = (x_data[ix_Y_True].sum(axis=0)+smoothing_p) / (sum(Y_data==True)+smoothing_p*10)
p_x_y_false = (p_x_y_true-1)*(-1)

In [ ]:
x_test = [0,1,0,0,0,1,0,0,1,0]
p_out_y_t = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_true) ]
p_out_y_true = 1

for i in p_out_y_t:
    p_out_y_true*=i

In [ ]:
p_out_y_true

In [ ]:
p_out_y_f = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_false) ]
p_out_y_false = 1

for i in p_out_y_f:
    p_out_y_false*=i

In [ ]:
p_out_y_false

In [ ]:
p_out_y_f = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_false) ]
p_out_y_false = 1

for i in p_out_y_f:
    p_out_y_false*=i

In [ ]:
p_out_y_false

In [ ]:
import math
 
p_y_true_test =  P_Y_True*p_out_y_true/(P_Y_True*p_out_y_true+P_Y_False*p_out_y_false)
p_y_false_test = P_Y_False*p_out_y_false/(P_Y_True*p_out_y_true+P_Y_False*p_out_y_false)

In [ ]:
p_y_true_test < p_y_false_test

In [ ]:
smoothing_p = 20

p_x_y_true = (x_data[ix_Y_True].sum(axis=0)+smoothing_p) / (sum(Y_data==True)+smoothing_p*10)
p_x_y_false = (p_x_y_true-1)*(-1)

In [ ]:
x_test = [0,1,0,0,0,1,0,0,1,0]

In [ ]:
p_out_y_t = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_true) ]
p_out_y_true = 1

for i in p_out_y_t:
    p_out_y_true*=i

In [ ]:
p_out_y_true

In [ ]:
p_out_y_f = [ p if x_test[n] == 1 else 1-p for n, p in enumerate(p_x_y_false) ]
p_out_y_false = 1

for i in p_out_y_f:
    p_out_y_false*=i

In [ ]:
p_out_y_false

In [ ]:
import math

p_y_true_test =  P_Y_True*p_out_y_true/(P_Y_True*p_out_y_true+P_Y_False*p_out_y_false)
p_y_false_test = P_Y_False*p_out_y_false/(P_Y_True*p_out_y_true+P_Y_False*p_out_y_false)

In [ ]:
# 위에서 얘기하였듯이 주어진 표본에서 다음 y의값이 1일 확률이 100%가 되면 실제로 현실에서 그런 경우는 없다고 봐도 무방하므로
# 인위적으로 최소한의 확률을 주는 smoothing을 하여보았다
# 그 결과 y가 true일 때의 확률이 100%가 아닌 특정 값으로 나왔지만 여전히 99%로 매우 높은 값을 가짐을 볼 수 있어 
# 좀더 높은 값으로 parameter를 20으로 주어 보았으나 여전히 99퍼센트로 y가 true일 확률이 매우 높은 것을 볼 수 있었다